In [3]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
#reader csvs
student_file = "Resources/students_complete.csv"
school_file = "Resources/schools_complete.csv"

studentdf = pd.read_csv(student_file)
schooldf = pd.read_csv(school_file)

#sample of student df
studentdf.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
#snips of Studen df
math_score = studentdf[["name","school","math_score"]]
read_score = studentdf[["name","school","reading_score"]]
mathby_grade = studentdf[["school","grade","math_score"]]
readby_grade = studentdf[["school","grade","reading_score"]]
math_ninth = mathby_grade.loc[(mathby_grade["grade"]=="9th")]


In [6]:
#snip of Passing Students
pass_math_snip = math_score.loc[math_score['math_score'] > 69]
pass_read_snip = read_score.loc[read_score['reading_score'] > 69]

In [7]:
#aggregate values
total_schools = schooldf['name'].count()
total_students = studentdf['name'].count()
total_budget = schooldf['budget'].sum()
avg_math_score = studentdf['math_score'].mean()
avg_reading_score = studentdf['reading_score'].mean()
pass_math = pass_math_snip['math_score'].count()
pass_read = pass_read_snip['reading_score'].count()
per_passing_math = (pass_math/total_students)*100
per_passing_reading = (pass_read/total_students)*100
overall_pass_rate = (per_passing_math + per_passing_reading)/2
ttl_budget = schooldf['budget'].sum()
ttl_students = studentdf['name'].count()

total_schools

15

In [8]:
#lay out district summary table using aggregate values
district_summary = pd.DataFrame({"Total Schools": [total_schools],
            "Total Students": [total_students],
            "Total Budget": [total_budget],
            "Average Math Score": [avg_math_score],
            "Average Reading Score": [avg_reading_score],
            "% Passing Math": [per_passing_math],
            "% Passing Reading": [per_passing_reading],
            "Overall Passing Rate": [overall_pass_rate]})

#reorder values for district summary table
district_summary = district_summary[["Total Schools",
                                     "Total Students",
                                     "Total Budget",
                                     "Average Math Score",
                                     "Average Reading Score",
                                     "% Passing Math",
                                     "% Passing Reading",
                                     "Overall Passing Rate"]]

In [9]:
#format values
district_summary["Total Schools"] = district_summary["Total Schools"].map("{0:,.0f}".format)
district_summary["Total Students"] = district_summary["Total Students"].map("{0:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${0:,.0f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{0:,.2f}%".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{0:,.2f}%".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{0:,.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{0:,.2f}%".format)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].map("{0:,.2f}%".format)
#print district_summary.to_string(index=False)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428",78.99%,81.88%,74.98%,85.81%,80.39%


In [10]:
school_snip = schooldf[["name", "type", "budget"]]
school_snip = school_snip.sort_values(["name"], ascending=True)
school_snip = school_snip.reset_index(drop=True)
school_snip = school_snip.rename(columns = {"name":"School",
                                           "type":"School Type",
                                           "budget":"Total School Budget"})
school_snip

,School,School Type,Total School Budget
0,Bailey High School,District,3124928
1,Cabrera High School,Charter,1081356
2,Figueroa High School,District,1884411
3,Ford High School,District,1763916
4,Griffin High School,Charter,917500
5,Hernandez High School,District,3022020
6,Holden High School,Charter,248087
7,Huang High School,District,1910635
8,Johnson High School,District,3094650
9,Pena High School,Charter,585858


In [79]:
#sort by
student_ttl = pd.DataFrame(studentdf.groupby("school")["name"].count())
student_ttl.reset_index(inplace=True)
student_ttl = student_ttl.rename(columns = {"name":"Total Students",
                                             "school":"School"})
student_ttl

student_budget = pd.merge(student_ttl, school_snip, on="School")
student_budget["Per Student Budget"] = student_budget["Total School Budget"] / student_budget["Total Students"]
student_budget = student_budget.drop(["Total Students"],axis=1)
student_budget = student_budget.drop(["School Type"],axis=1)
student_budget = student_budget.drop(["Total School Budget"],axis=1)
student_budget


,School,Per Student Budget
0,Bailey High School,628.0
1,Cabrera High School,582.0
2,Figueroa High School,639.0
3,Ford High School,644.0
4,Griffin High School,625.0
5,Hernandez High School,652.0
6,Holden High School,581.0
7,Huang High School,655.0
8,Johnson High School,650.0
9,Pena High School,609.0


In [7]:
mathpassdf = pd.DataFrame(pass_math_snip.groupby("school")["math_score"].count())

mathpassdf.reset_index(inplace=True)
mathpassdf.head()

,school,math_score
0,Bailey High School,3318
1,Cabrera High School,1749
2,Figueroa High School,1946
3,Ford High School,1871
4,Griffin High School,1371


In [8]:
readpassdf = pd.DataFrame(pass_read_snip.groupby("school")["reading_score"].count())
readpassdf.reset_index(inplace=True)
readpassdf
scoresmerge = pd.merge(readpassdf, mathpassdf, on="school")
scoresmerge = scoresmerge.rename(columns= {"school": "School",
                                          "reading_score": "Passed Reading",
                                          "math_score": "Passed Math"})
scoresmerge.head()

,School,Passed Reading,Passed Math
0,Bailey High School,4077,3318
1,Cabrera High School,1803,1749
2,Figueroa High School,2381,1946
3,Ford High School,2172,1871
4,Griffin High School,1426,1371


In [ ]:
math_avg = pd.DataFrame(studentdf.groupby("school")["math_score"].mean())
read_avg = pd.DataFrame(studentdf.groupby("school")["reading_score"].mean())

math_avg.reset_index(inplace=True)
read_avg.reset_index(inplace=True)

math_avg = math_avg.rename(columns = {"school":"School"})
read_avg = read_avg.rename(columns = {"school":"School"})

merge_df = pd.merge(math_avg, read_avg, on="School")
merge_df = pd.merge(merge_df, student_ttl, on="School")
merge_df = pd.merge(merge_df, scoresmerge, on="School")

merge_df = merge_df.rename(columns = {"math_score": "Avg Math Score",
                                     "reading_score": "Avg Reading Score"})

merge_df["% Passing Math"] = merge_df["Passed Math"] / merge_df["Total Students"] * 100
merge_df["% Passing Reading"] = merge_df["Passed Reading"] / merge_df["Total Students"] * 100
merge_df["% Overall Passing Rate"] = merge_df[["% Passing Math","% Passing Reading"]].mean(axis=1)
school_summary = merge_df.drop(["Passed Reading","Passed Math"], axis=1)
school_summary = pd.merge(school_summary, school_snip, on = "School")
school_summary = pd.merge(school_summary, student_budget, on = "School")
school_summary.head()

In [21]:
botfive = school_summary.nsmallest(5,'% Overall Passing Rate')
botfive.reset_index(inplace=True)
botfive.drop(['index'],axis=1)

,School,Avg Math Score,Avg Reading Score,Total Students,% Passing Math,% Passing Reading,% Overall Passing Rate,School Type,Total School Budget
0,Rodriguez High School,76.842711,80.744686,3999,66.366592,80.220055,73.293323,District,2547363
1,Figueroa High School,76.711767,81.158020,2949,65.988471,80.739234,73.363852,District,1884411
2,Huang High School,76.629414,81.182722,2917,65.683922,81.316421,73.500171,District,1910635
3,Johnson High School,77.072464,80.966394,4761,66.057551,81.222432,73.639992,District,3094650
4,Ford High School,77.102592,80.746258,2739,68.309602,79.299014,73.804308,District,1763916


In [22]:
topfive = school_summary.nlargest(5,'% Overall Passing Rate')
topfive.reset_index(inplace=True)
topfive.drop(['index'],axis=1)

,School,Avg Math Score,Avg Reading Score,Total Students,% Passing Math,% Passing Reading,% Overall Passing Rate,School Type,Total School Budget
0,Cabrera High School,83.061895,83.975780,1858,94.133477,97.039828,95.586652,Charter,1081356
1,Thomas High School,83.418349,83.848930,1635,93.272171,97.308869,95.290520,Charter,1043130
2,Pena High School,83.839917,84.044699,962,94.594595,95.945946,95.270270,Charter,585858
3,Griffin High School,83.351499,83.816757,1468,93.392371,97.138965,95.265668,Charter,917500
4,Wilson High School,83.274201,83.989488,2283,93.867718,96.539641,95.203679,Charter,1319574
